In [ ]:


!pip install --no-deps bitsandbytes accelerate xformers peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 8.7 MB/s eta 0:00:00


In [ ]:
from unsloth import FastLanguageModel
import torch

from google.colab import userdata


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.10: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "How to do an startup"}
]

formatted_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

print(formatted_text)


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

How to do an startup<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [ ]:
def formatting_prompts_func(examples):
  convos = examples["conversations"]
  texts = [tokenizer.apply_chat_template(convo,tokenize=False, add_generation_prompt=False) for convo in convos]
  return { "text": texts,}
  pass

from datasets import load_dataset

dataset = load_dataset("pookie3000/pg_chat", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)
dataset

In [ ]:
for i, sample in enumerate(dataset):
  print(f"\n--------- Sample { i+ 1} ----")
  print(sample["text"])
  if i >= 1:
    break


--------- Sample 1 ----
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

What is your name?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Nice to meet you! My name is Paul Graham, and I'm delighted to make your acquaintance.<|eot_id|>

--------- Sample 2 ----
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

What's your name?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Nice to meet you! My name is Paul Graham, and I'm delighted to make your acquaintance.<|eot_id|>


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from peft import LoraConfig
from unsloth import is_bfloat16_supported

target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]


model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = target_modules,
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None
)


Unsloth 2025.8.10 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
# Apply Lora

target_models = [
  "q_proj","k_proj","v_proj","o_proj","gate_proj", "up_proj","down_proj"
]

train_embeddings = False

if train_embeddings:
  target_modules = target_models + ["lm_head"]


from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    max_seq_length = 2048,
    train_dataset=dataset,
    dataset_text_field = "text",
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        num_train_epochs = 2,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        tf32 = False,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none"
  )
)

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 484 | Num Epochs = 2 | Total steps = 122
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,entropy
1,2.084300,0
2,1.869900,No Log
3,1.891500,No Log
4,1.621800,No Log
5,1.558000,No Log
6,1.479900,No Log
7,1.394100,No Log
8,1.400300,No Log
9,1.373400,No Log
10,1.363600,No Log


In [ ]:
FastLanguageModel.for_inference(model)

messages = [
    { "role": "user", "content": "How to do a startup?"}
]

inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True,
                                       return_tensors="pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 200, use_cache = True)



The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

How to do a startup?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"Ah, you want to know how to do a startup, huh? Well, let me tell you, it's not for the faint of heart. But if you're willing to put in the hard work, I'll give you the lowdown. First, you gotta be willing to take risks. Like, really big risks. We're talking about pouring your heart and soul into something that might just fail. But if you're not willing to take that risk, then you're not going to make it in this game.

Next, you gotta be passionate about what you're doing. I mean, really passionate. Like, you can't sleep at night because you're thinking about it passionate. Because if you're not, then you're just going to burn out.

And then, you gotta be willing to learn. Like, really learn. Because the truth is, you don't know what you